In [ ]:
%matplotlib ipympl

#original
#import xanes_math as xm
#import xanes_analysis as xa
import xanes_analysis_gauss_s1 as xa

import h5py, tifffile
from pathlib import Path
import os, sys
import matplotlib.pyplot as plt
import numpy as np
from importlib import reload
import skimage.morphology as skm
import scipy.ndimage as spn
from numpy import savetxt
xa = reload(xa)
plt.rcParams['figure.figsize'] = [14, 20]
print('\033[04m\033[01m\033[34m     Section 1 finished     ')

In [ ]:
# edge_offset from Co
edge_offset_2_Co = 8.333 - 8.333
# estimated edge energy
edge_eng = 8.333 + edge_offset_2_Co
# end poit of the pre-edge relative to the edge_eng in keV
pre_ee = -0.05
# start point of the post-edge relative to the edge_eng in keV
post_es = 0.1
# how many times of the edge jump magnitude should be compared to the pre-edge standard deviation
edge_jump_threshold = 3
# how much should the pre_edge be offset up for validating if the post edge trend is in a reasonable range
# this is a factor to the pre-edge deviation
pre_edge_threshold = 3.5
# define an energy range for 0.5 absorption postion fitting
ep_eng_s = 8.335 + edge_offset_2_Co
ep_eng_e = 8.350 + edge_offset_2_Co
# define an energy range for whiteline peak postion fitting
#wl_eng_s = 8.345 + edge_offset_2_Co + 0.000
#wl_eng_e = 8.355 + edge_offset_2_Co - 0.000

# define an energy range for edge_pos display
ep_vmin = 8.338 + edge_offset_2_Co
ep_vmax = 8.348 + edge_offset_2_Co
# define an energy range for whiteline display
wl_vmin = 8.340 + edge_offset_2_Co
wl_vmax = 8.360 + edge_offset_2_Co
# define path and file name to save xanes analysis results; if you use the default path and name as below,
# you don't need to change anything. otherwise, give your full path and file name below.
#out_fn = os.path.join(str(Path(fn_template).parent), 'xanes_analysis_' + str(Path(fn_template).name)).format(scan_id)
#print(out_fn)
print('\033[04m\033[01m\033[34m     Section 2 finished     ')

In [ ]:
fn = []
eng = []
fn = np.array(fn)
eng = np.array(eng)

f_path ='/run/media/VTLinlab/Elements/Rod3/Rod3_200_second/'
f_name = '3D_trial_reg_scan_id_27297-27317_2020-04-26-11-38-07.h5'

#change h5 file
fn = f_path+f_name

f = h5py.File(fn, 'r')
eng = np.array(f['/registration_results/reg_results/eng_list'])
imgs_all = f['/registration_results/reg_results/registered_xanes3D']
ny = imgs_all.shape[2]
nx = imgs_all.shape[3]
nz = imgs_all.shape[1]

print(imgs_all.shape)

In [ ]:
# numbers for rebin need be Divisible
rebin_factor = 3
imgs_4rebin= imgs_all[:, 0:nz//rebin_factor*rebin_factor, 0:ny//rebin_factor*rebin_factor, 0:nx//rebin_factor*rebin_factor]

# rebin images
def rebin(arr, new_shape):
    shape = (new_shape[0], arr.shape[0] // new_shape[0],
             new_shape[1], arr.shape[1] // new_shape[1],
             new_shape[2], arr.shape[2] // new_shape[2],
             new_shape[3], arr.shape[3] // new_shape[3])
    return arr.reshape(shape).mean(-1).mean(-2).mean(-3).mean(-4)
ny = imgs_4rebin.shape[2]
nx = imgs_4rebin.shape[3]
nz = imgs_4rebin.shape[1]
print(imgs_4rebin.shape)

# rebin all images
rebin_imgs = np.ndarray([imgs_all.shape[0], int(nz/rebin_factor), int(ny/rebin_factor), int(nx/rebin_factor)])
rebin_imgs = rebin(imgs_4rebin, [rebin_imgs.shape[0], rebin_imgs.shape[1], rebin_imgs.shape[2], rebin_imgs.shape[3]])
print(rebin_imgs.shape)


# exclude the last energy point
#eng = eng [:-1]
#imgs = np.ndarray([rebin_imgs.shape[0]-1, rebin_imgs.shape[2], rebin_imgs.shape[3]])
#xanes3d = np.ndarray([rebin_imgs.shape[1], rebin_imgs.shape[2], rebin_imgs.shape[3]])

#print('Your results will be saved in the file', '\"'+out_fn+'\"')
print('\033[04m\033[01m\033[34m     Section 3 finished     ')

In [ ]:
# save rebin images
fn_rebin = f_path+"rebin_"+f_name
hf = h5py.File(fn_rebin, 'w')
hf.create_dataset('rebin_3x3x3', data=rebin_imgs)
hf.close()
print('rebin data saved')

In [ ]:
# read rebin images
fn_rebin = f_path+"rebin_"+f_name
hf = h5py.File(fn_rebin, 'r')
rebin_imgs = hf['/rebin_3x3x3']
print(rebin_imgs.shape)

In [ ]:
# make mask

rebin_mask = np.ndarray([rebin_imgs.shape[1], rebin_imgs.shape[2], rebin_imgs.shape[3]])
imgs = np.ndarray([rebin_imgs.shape[0], rebin_imgs.shape[2], rebin_imgs.shape[3]])

## rebin_mask with threshold_ratio
#mask_threshold_ratio = 0.25
#mask_dilation = 3
#for ii in range(rebin_imgs.shape[1]):
#    imgs[idx] = rebin_imgs[idx, ii]
#    mask_threshold = imgs[idx].min().min()+ (imgs[idx].max().max()-imgs[idx].min().min())*mask_threshold_ratio
#    rebin_mask[ii] = skm.binary_dilation((spn.gaussian_filter(imgs[idx],mask_dilation) > mask_threshold).astype(np.uint8), np.ones([mask_dilation,mask_dilation])).astype(np.uint8)[:]

## rebin_mask with fixed threshold
mask_threshold = 0.0015
mask_dilation = 2

idx = int(rebin_imgs.shape[0]/2)
for ii in range(rebin_imgs.shape[1]):
    imgs[idx] = rebin_imgs[idx, ii]
#    rebin_mask[ii] = skm.binary_dilation((imgs[idx] > mask_threshold).astype(np.uint8), np.ones([mask_dilation,mask_dilation])).astype(np.uint8)[:]
    rebin_mask[ii] = skm.binary_dilation((spn.gaussian_filter(imgs[idx],mask_dilation) > mask_threshold).astype(np.uint8), np.ones([mask_dilation,mask_dilation])).astype(np.uint8)[:]

## plot top, middle, bottom slices of the mask * image

ii = int(0.05*rebin_imgs.shape[1])
fig = plt.figure()
ax3 = fig.add_subplot(1, 3, 1)
ax3.set_title('image', fontdict={'fontsize':12})
ax3.imshow(rebin_imgs[idx, ii, :, :])
ax4 = fig.add_subplot(1, 3, 2)
ax4.set_title('mask', fontdict={'fontsize':12})
ax4.imshow(rebin_mask[ii, :, :])
ax5 = fig.add_subplot(1, 3, 3)
ax5.set_title('image x mask', fontdict={'fontsize':12})
ax5.imshow(rebin_imgs[idx, ii, :, :]*rebin_mask[ii, :, :])

ii = int(0.5*rebin_imgs.shape[1])
fig2 = plt.figure()
ax32 = fig2.add_subplot(1, 3, 1)
ax32.set_title('image', fontdict={'fontsize':12})
ax32.imshow(rebin_imgs[idx, ii, :, :])
ax42 = fig2.add_subplot(1, 3, 2)
ax42.set_title('mask', fontdict={'fontsize':12})
ax42.imshow(rebin_mask[ii, :, :])
ax52 = fig2.add_subplot(1, 3, 3)
ax52.set_title('image x mask', fontdict={'fontsize':12})
ax52.imshow(rebin_imgs[idx, ii, :, :]*rebin_mask[ii, :, :])

ii = int(0.95*rebin_imgs.shape[1])
fig3 = plt.figure()
ax33 = fig3.add_subplot(1, 3, 1)
ax33.set_title('image', fontdict={'fontsize':12})
ax33.imshow(rebin_imgs[idx, ii, :, :])
ax43 = fig3.add_subplot(1, 3, 2)
ax43.set_title('mask', fontdict={'fontsize':12})
ax43.imshow(rebin_mask[ii, :, :])
ax53 = fig3.add_subplot(1, 3, 3)
ax53.set_title('image x mask', fontdict={'fontsize':12})
ax53.imshow(rebin_imgs[idx, ii, :, :]*rebin_mask[ii, :, :])

plt.show()

In [ ]:
# plot xanes for slice ii at specific point (x,y)
ii = int(0.5*rebin_imgs.shape[1])

for jj in range(rebin_imgs.shape[0]):
    imgs[jj] = rebin_imgs[jj, ii]

x=220
y=180

fig = plt.figure()
ax1 = fig.add_subplot(1, 1, 1)
#ax1.plot(eng, imgs[:, y, x])
ax1.plot(eng, imgs[:, y, x]*rebin_mask[ii, y, x])
#ax1.xlim([8.347, 8.355])
plt.show()

In [ ]:
# whiteline fitting of rebin_imgs, poly2, with mask
#change out file name!!
poly_order = 2
out_fn = f_path+"Whiteline_Fitting_gauss_s1_rebin_mask_poly2_"+f_name

wl_eng_s = 8.344 + edge_offset_2_Co + 0.000
wl_eng_e = 8.355 + edge_offset_2_Co - 0.000

#wl_eng_s = 8.344 + edge_offset_2_Co + 0.000
#wl_eng_e = 8.353 + edge_offset_2_Co - 0.000

#wl_eng_s = 8.346 + edge_offset_2_Co + 0.000
#wl_eng_e = 8.355 + edge_offset_2_Co - 0.000

imgs = np.ndarray([rebin_imgs.shape[0], rebin_imgs.shape[2], rebin_imgs.shape[3]])
xanes3d = np.ndarray([rebin_imgs.shape[1], rebin_imgs.shape[2], rebin_imgs.shape[3]])
xana = xa.xanes_analysis(imgs, eng, edge_eng, pre_ee=pre_ee, post_es=post_es, 
                         edge_jump_threshold=edge_jump_threshold, pre_edge_threshold=pre_edge_threshold)

for ii in range(rebin_imgs.shape[1]):
    for jj in range(rebin_imgs.shape[0]):
        imgs[jj] = rebin_imgs[jj, ii]

    xana.spectrum[:] = imgs[:]
    xana.fit_whiteline(wl_eng_s, wl_eng_e, poly_order)
    xanes3d[ii] = xana.wl_pos[:]*rebin_mask[ii]
#    xanes3d[ii] = xana.whiteline_pos[:]

xana.save_results(out_fn, dtype='3D_XANES', **{'whiteline': xanes3d})
print(f_path)
print("Whiteline_Fitting_gauss_s1_rebin_mask_poly2_"+f_name)

In [ ]:
# whiteline fitting of rebin_imgs poly3
#change out file name
poly_order = 3
out_fn = f_path+"Whiteline_Fitting_gauss_s1_rebin_mask_poly3_"+f_name

wl_eng_s = 8.344 + edge_offset_2_Co + 0.000
wl_eng_e = 8.355 + edge_offset_2_Co - 0.000

#wl_eng_s = 8.344 + edge_offset_2_Co + 0.000
#wl_eng_e = 8.353 + edge_offset_2_Co - 0.000

#wl_eng_s = 8.346 + edge_offset_2_Co + 0.000
#wl_eng_e = 8.355 + edge_offset_2_Co - 0.000

imgs = np.ndarray([rebin_imgs.shape[0], rebin_imgs.shape[2], rebin_imgs.shape[3]])
xanes3d = np.ndarray([rebin_imgs.shape[1], rebin_imgs.shape[2], rebin_imgs.shape[3]])

xana = xa.xanes_analysis(imgs, eng, edge_eng, pre_ee=pre_ee, post_es=post_es, 
                         edge_jump_threshold=edge_jump_threshold, pre_edge_threshold=pre_edge_threshold)

for ii in range(rebin_imgs.shape[1]):
    for jj in range(rebin_imgs.shape[0]):
        imgs[jj] = rebin_imgs[jj, ii]

    xana.spectrum[:] = imgs[:]
    xana.fit_whiteline(wl_eng_s, wl_eng_e, poly_order)
    xanes3d[ii] = xana.wl_pos[:]*rebin_mask[ii]
#    xanes3d[ii] = xana.whiteline_pos[:]

xana.save_results(out_fn, dtype='3D_XANES', **{'whiteline': xanes3d})
print(f_path)
print("Whiteline_Fitting_gauss_s1_rebin_mask_poly3_"+f_name)

In [ ]:
hf.close()
f.close()